# 04 - Train Model

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


#notebook settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
data = pd.read_csv(os.path.join('..','data','processed','data_for_model.csv'))

print(data.shape)

data.head()

(7389, 95)


home_team   away_team full_time_result  day_of_week  month  \
0    aston villa      bolton                D            5      8   
1        everton  man united                A            5      8   
2         fulham  birmingham                D            5      8   
3       man city   west brom                D            5      8   
4  middlesbrough   liverpool                D            5      8   

   day_of_week_sin  day_of_week_cos  month_sin  month_cos  \
0        -0.866025              0.5  -0.866025       -0.5   
1        -0.866025              0.5  -0.866025       -0.5   
2        -0.866025              0.5  -0.866025       -0.5   
3        -0.866025              0.5  -0.866025       -0.5   
4        -0.866025              0.5  -0.866025       -0.5   

   home_roll_3_avg_home_total_goals  away_roll_3_avg_home_total_goals  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   home_roll_3_avg_away_total_goals  away_roll_3_avg_away_total_goals  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   home_roll_3_avg_home_total_shots  away_roll_3_avg_home_total_shots  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   home_roll_3_avg_away_total_shots  away_roll_3_avg_away_total_shots  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   home_roll_3_avg_home_shots_on_target  away_roll_3_avg_home_shots_on_target  \
0                                   0.0                                   0.0   
1                                   0.0                                   0.0   
2                                   0.0                                   0.0   
3                                   0.0                                   0.0   
4                                   0.0                                   0.0   

   home_roll_3_avg_away_shots_on_target  away_roll_3_avg_away_shots_on_target  \
0                                   0.0                                   0.0   
1                                   0.0                                   0.0   
2                                   0.0                                   0.0   
3                                   0.0                                   0.0   
4                                   0.0                                   0.0   

   home_roll_3_avg_home_fouls  away_roll_3_avg_home_fouls  \
0                         0.0                         0.0   
1                         0.0                         0.0   
2                         0.0                         0.0   
3                         0.0                         0.0   
4                         0.0                         0.0   

   home_roll_3_avg_away_fouls  away_roll_3_avg_away_fouls  \
0                         0.0                         0.0   
1     

In [7]:
data.columns

Index(['home_team', 'away_team', 'full_time_result', 'day_of_week', 'month',
       'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
       'home_roll_3_avg_home_total_goals', 'away_roll_3_avg_home_total_goals',
       'home_roll_3_avg_away_total_goals', 'away_roll_3_avg_away_total_goals',
       'home_roll_3_avg_home_total_shots', 'away_roll_3_avg_home_total_shots',
       'home_roll_3_avg_away_total_shots', 'away_roll_3_avg_away_total_shots',
       'home_roll_3_avg_home_shots_on_target',
       'away_roll_3_avg_home_shots_on_target',
       'home_roll_3_avg_away_shots_on_target',
       'away_roll_3_avg_away_shots_on_target', 'home_roll_3_avg_home_fouls',
       'away_roll_3_avg_home_fouls', 'home_roll_3_avg_away_fouls',
       'away_roll_3_avg_away_fouls', 'home_roll_3_avg_home_corners',
       'away_roll_3_avg_home_corners', 'home_roll_3_avg_away_corners',
       'away_roll_3_avg_away_corners', 'home_roll_3_avg_home_yellow_cards',
       'away_roll_3_avg_home_yellow

In [17]:
!pipenv install keras tensorflow

Installing keras...
✔ Installation Succeeded
Installing tensorflow...
✔ Installation Succeeded
Installing dependencies from Pipfile.lock (e2088c)...
All dependencies are now up-to-date!
Upgrading keras, tensorflow in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠴ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠸ Locking packages...
Installing dependencies from Pipfile.lock (e2088c)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (e2088c)...


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from scikeras.wrappers import KerasClassifier 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from scikeras.wrappers import KerasClassifier 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



# Extract unique teams from both columns
all_teams = pd.concat([data['home_team'], data['away_team']]).unique()

# Count the number of unique teams
num_teams = len(all_teams)

print(f"Number of unique teams: {num_teams}") 


# encode teams
le = LabelEncoder()
data['home_team'] = le.fit_transform(data['home_team']) 
data['away_team'] = le.fit_transform(data['away_team']) 

# encode result
le_y = LabelEncoder()
data['full_time_result'] = le_y.fit_transform(data['full_time_result'])

# Split the data into X and y
X = data.drop(columns=['full_time_result'])
y = data['full_time_result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the model with a fixed embedding_dim of 32
def create_model():
    model = Sequential()
    model.add(Embedding(num_teams, 32, input_length=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Add softmax activation

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

Number of unique teams: 44


/home/ruifspinto/.local/share/virtualenvs/mlzoomcamp2024-final-project-_5cYWqbG/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/ruifspinto/.local/share/virtualenvs/mlzoomcamp2024-final-project-_5cYWqbG/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


AttributeError: Sequential model 'sequential_2' has no defined outputs yet.